In [24]:
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import callbacks
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.callbacks import TensorBoard
import datetime
import numpy as np

In [25]:
# the data, shuffled and split between train and test sets
def load_data(dataset):
    # Load
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    # Reshape X
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    n_channels = x_train.shape[3]
    if K.image_data_format() == 'channels_first':
        s = [n_channels, img_rows, img_cols]
    else:
        s = [img_rows, img_cols, n_channels]
    x_train = x_train.reshape(x_train.shape[0], *s)
    x_test = x_test.reshape(x_test.shape[0], *s)
    # Rescale X
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Reshape y
    num_classes = len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test), img_rows, img_cols

In [26]:
def get_tensorboard(name=None):
    name = name or 'Graph'
    return TensorBoard(
        log_dir='./{0}/{1}'.format(name, get_timestamp()), histogram_freq=0,
        write_graph=True, write_images=False
    )


def get_timestamp():
    return '{:%Y-%m-%d-%H-%M-%S}'.format(datetime.datetime.now())

In [27]:
(x_train, y_train), (x_test, y_test), img_rows, img_cols = load_data(cifar10)

In [28]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [29]:
linear = Sequential()
linear.add(Flatten(input_shape=x_train.shape[1:]))
linear.add(Dense(units=10, activation='softmax'))

In [30]:
small_cnn = Sequential()

small_cnn.add(Conv2D(32, (5, 5), activation='relu', input_shape=x_train.shape[1:], padding="same"))
small_cnn.add(Conv2D(32, (5, 5), activation='relu',  padding="same", ))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(64, (3, 3), activation='relu',  padding="same"))
small_cnn.add(Conv2D(64, (3, 3), activation='relu',  padding="same"))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(128, (3, 3), activation='relu',  padding="same"))
small_cnn.add(Conv2D(128, (3, 3), activation='relu',  padding="same"))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Flatten())

small_cnn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
small_cnn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

small_cnn.add(Dense(units=10, activation='softmax'))

In [31]:
#model = linear
model = small_cnn

#Set up tensorboard
tb = get_tensorboard(name="Graph1")

# Set up Data Generator
datagen = ImageDataGenerator(rotation_range=90)
datagen.fit(x_train)
datagen.flow(x_train, y_train)

In [32]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
n_train    = 6000
batch_size = 64
epochs     = 10

model.fit(x_train[:n_train], y_train[:n_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[:n_train], y_test[:n_train]), callbacks=[tb])

Train on 6000 samples, validate on 6000 samples
Epoch 1/10
6000/6000 [==============================] - 165s - loss: 3.4135 - acc: 0.1453 - val_loss: 2.9306 - val_acc: 0.1490
Epoch 2/10
6000/6000 [==============================] - 153s - loss: 2.6540 - acc: 0.2183 - val_loss: 2.4633 - val_acc: 0.2423
Epoch 3/10
6000/6000 [==============================] - 149s - loss: 2.3621 - acc: 0.2582 - val_loss: 2.2714 - val_acc: 0.2978
Epoch 4/10
6000/6000 [==============================] - 163s - loss: 2.2026 - acc: 0.2853 - val_loss: 2.1031 - val_acc: 0.3267
Epoch 5/10
6000/6000 [==============================] - 156s - loss: 2.0715 - acc: 0.3230 - val_loss: 2.1018 - val_acc: 0.3160
Epoch 6/10
6000/6000 [==============================] - 151s - loss: 1.9584 - acc: 0.3535 - val_loss: 1.9880 - val_acc: 0.3522
Epoch 7/10
6000/6000 [==============================] - 152s - loss: 1.8732 - acc: 0.3777 - val_loss: 1.8211 - val_acc: 0.3932
Epoch 8/10
  64/6000 [..............................] - ETA: 12

In [23]:
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', acc)

Test accuracy: 0.4407
